# Exploring the Relationship Between the Meta Scores of Films and the Reviews of the General Audiance


## Introduction:
IMDB is an online database for films, television series, podcasts, and other media. This project will be working with movie data from IMDB. The data set of interest contains content data - including cast, production crew, plot summary, and scores. IMDB presents two kinds of scores; IMDB scores which are user-generated averages of reviews submitted to IMDB by the general audience, and Meta Scores which are critic-generated, representing the views of professionals who analyze films.

The goal of this project is to answer the following question:
"How accurate are the Meta Scores of films in predicting the reception of the general audience?"


The data set is from Kaggle, and includes data from the top 1000 movies based on the IMDB score. The data set includes the following columns:
* Series_Title: the name of the film
* Released_year: year it was released
* Certificate: certificate earned by the movie
* Runtime - total runtime
* Genre - list of genres the film falls into
* IMDB_Rating - the IMDB rating given by IMDB user reviews
* Overview - plot summary
* Meta_score - meta score of the movie determined by movie critics
* Director - name of the director of the movie
* Star1, Star2, Star3, Star4 - names of the stars of the movie in order of significance
* No_of_votes - number of reviews on IMDB
* Gross - how much money the movie earned

Dataset Origin: https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

## Methods:
#### 1. Import Libraries
Tidyverse, dplyr, and tidymodels libraries were imported for data manipulation and visualization.

In [ ]:
library(tidyverse)
library(dplyr)
library(tidymodels)
options(repr.matrix.max.rows = 8)

#### 2. Read data from repository
The data was downloaded from: https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows
then added to this project's repository. The following cell reads the data from the repository as a CSV file and filters for films released after 1970, as these films will represent modern conditions better.

In [ ]:
### Reading from Github
url <- "https://github.com/anh-dong/dsci-100-2023w1-group-33/blob/main/data/imdb_top_1000.csv?raw=true"
movies <- read_csv(url) |>
    filter(Released_Year > 1970) |>
    select(IMDB_Rating, Meta_score)
    # na.omit()

#### 3. Create Training Data
The raw data is then split into a training and testing set of 75% and 25% of the raw data respectively.

In [ ]:
set.seed(1234)

movies_split <- initial_split(movies, prop = 0.75, strata = Meta_score)
movies_training <- training(movies_split)
movies_testing <- testing(movies_split)

#### 4. Summary Table
The training data was summarized by first normalizing and counting each normalized value for Meta_score and IMDB_Rating.

In [ ]:
movies_summary <- movies_training |>
    mutate(Meta_score = round(scale(Meta_score), 0), IMDB_Rating = round(scale(IMDB_Rating), 0))

movies_table_Meta_scores <- movies_summary |>
    group_by(Meta_score) |>
    summarize(Meta_score_count = n()) |>
    rename(normalized_value = Meta_score)

movies_table_IMDB_Rating <- movies_summary |>
    group_by(IMDB_Rating) |>
    summarize(IMDB_Rating_count = n()) |>
    rename(normalized_value = IMDB_Rating)

joined <- left_join(movies_table_Meta_scores, movies_table_IMDB_Rating) |>
    mutate(IMDB_Rating_count = replace(IMDB_Rating_count, is.na(IMDB_Rating_count), 0))
joined

##### Table 1:
Counts of normalized values for Meta_score and IMDB_Rating for the training data set.

#### 5. Summary Visual
The distribution of the normalized data was then visualized by plotting them on

In [ ]:
Meta_score_hist <- movies_summary |>
    ggplot(aes(Meta_score)) +
        geom_histogram(binwidth = 1) +
        xlab("Normalized Meta Score") +
        ylab("Count") +
        ggtitle("Distribution of Normalized Meta Scores")
Meta_score_hist

##### Figure 1:
Histogram of normalized IMDB Ratings using geom_histogram with a binwidth of 1.

In [ ]:
IMDB_Review_hist <- movies_summary |>
    ggplot(aes(IMDB_Rating)) +
        geom_histogram(binwidth = 1) +
        xlab("Normalized IMDB Rating") +
        ylab("Count") +
        ggtitle("Distribution of Normalized IMDB Ratings") +
        
IMDB_Review_hist

##### Figure 2: 
Histogram of normalized IMDB Ratings with a binwidth of 1.

#### 6. Linear Regression
To best determine the relationship between the IMDB Rating and the Meta Score a linear regression was performed using Meta_score as the predictor and IMDB Rating as the response variable. The training data was fit to this workflow to predict parameters for the y-intercept and slope of the regression line.

In [ ]:
lm_spec <- linear_reg() |>
  set_engine("lm") |>
  set_mode("regression")

lm_recipe <- recipe(IMDB_Rating ~ Meta_score, data = movies_training)

lm_fit <- workflow() |>
  add_recipe(lm_recipe) |>
  add_model(lm_spec) |>
  fit(data = movies_training)
lm_fit

#### 7. Visualization of Training Data and Line of Best Fit
The linear regression line was then plotted against the training data to visualize how well it fits the data.

In [ ]:
movies_preds <- lm_fit |>
   predict(movies_training) |>
   bind_cols(movies_training)

lm_predictions <- movies_preds |>
    ggplot(aes(x = Meta_score, y = IMDB_Rating)) +
        geom_point(alpha = 0.4) +
        geom_line(
            mapping = aes(x = Meta_score, y = .pred), 
            color = "blue") +
        ggtitle("Linear Regression of Meta Score vs. IMDB Rating Score")+
        xlab("Meta Score")+
        ylab("IMDB Rating")
lm_predictions

##### Figure 3: 
Scatter plot of IMDB Rating vs Meta Score. The IMDB Rating is a value out of 10 representing user reviews, while the Meta Score is a value out of 100 representing critic scores. The line of best fit in blue was calculated using linear regression fit against the data.

#### 8. Root Mean Squared Error (RMSE)
RMSE was calculated by binding our predicted values to our training data set and calculating metrics with IMDB_Rating as out truth value.

In [ ]:
lm_training_results <- lm_fit |>
         predict(movies_training) |>
         bind_cols(movies_training) |>
         metrics(truth = IMDB_Rating, estimate = .pred)

lm_rmse <- lm_training_results |>
          filter(.metric == "rmse") |>
          select(.estimate) |>
          pull()

lm_rmse

#### 9. Testing
The testing data set was then compared to their predictions based on our model and plotted against the line of best fit calculated above.

In [ ]:
test_preds <-  lm_fit |>
   predict(movies_testing) |>
   bind_cols(movies_testing)

lm_predictions_test <- test_preds |>
     ggplot(aes(x = Meta_score, y =IMDB_Rating )) +
         geom_point(alpha = 0.4) +
         geom_line(
             mapping = aes(x = Meta_score, y = .pred), 
             color = "blue") +
        xlab("Meta Score")+
        ylab("IMDB Rating")+ 
        ggtitle("Meta Score vs IMDB Rating for the Test set") +
         theme(text = element_text(size = 15))

lm_predictions_test

##### Figure 4: 
The training data set is plotted with IMDB Rating against Meta Score. IMDB Rating are from reviews of a general audience as a score out of 10. The Meta Score is an aggregate critic score out of 100. The blue line was determined through a linear regression of our training data set.

#### 10. Root Mean Sqaured Precent Error (RMSPE)
The RMSPE was calculated by using our model to predict IMDB_Rating for our testing data set. This metric is an an estimate for our standard deviation of our testing data set against our model.

In [ ]:
lm_test_results <- lm_fit |>
         predict(movies_testing) |>
         bind_cols(movies_testing) |>
         metrics(truth = IMDB_Rating, estimate = .pred)

lm_rmspe <- lm_test_results |>
          filter(.metric == "rmse") |>
          select(.estimate) |>
          pull()

lm_rmspe

## Expected Outcomes and Significance:
It is expected that the Meta Scores and audience reception will have a weak positive relationship. Critics are trying to give audiences an accurate expectation on the film based on their ratings so they should correlate positively with each other. However, critics and general audiences often look for different things in their ratings, producing more variability between the two ratings, potentially causing a weaker relationship. This relationship will allow audiences to understand how a meta score should factor into their decision to see a movie before any general audience ratings are available.

## Resources:

How Meta Scores are computed:

How do you compute metascores? – metacritic support. Metacritic Support. (2023, October). https://metacritichelp.zendesk.com/hc/en-us/articles/14478499933079-How-do-you-compute-METASCORES- 



Factors that influence Meta Scores (critics): 

Gruber, M. (2022, April 20). How critics produce a film analysis. Ready Steady Cut. https://readysteadycut.com/2022/04/20/how-critics-produce-a-film-analysis/ 



Factors that effect IMDB score (a report): 

Türker, B. (2021, August 17). Eda project - what are the factors affecting IMDB ratings?. Medium. https://medium.com/i%CC%87stanbuldatascienceacademy/eda-project-what-are-the-factors-affecting-imdb-ratings-e91f41396c89 



Inconsistency in how movie critics and general viewers judge films:

Collazo, M. (2014, April 30). How movie critics and moviegoers view films differently. The Artifice. https://the-artifice.com/movie-critics-and-moviegoers-view-films-differently/ 


Data Set Origin:

Shankhdhar, H. (2021, February 1). IMDB movies dataset. Kaggle. https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows 

